In [1]:
import pandas as pd
import glob,os

In [74]:
#path = os.getcwd()
extension = 'csv'

def getFiles():
    return glob.glob('*.{}'.format(extension))

def prepareData():
    files = getFiles()
    list_of_df = []
    for file in files :
        data = pd.read_csv(file, index_col='Unnamed: 0')
        list_of_df.append(data)
    df = pd.concat(x for x in list_of_df)
    return df
df = prepareData()
df = df.sort_values('date')

In [75]:
#df_group = df.groupby(['PERMNO','date'])[['SHRCD', 'EXCHCD', 'PRC', 'VOL', 'RET','SHROUT']].agg(sum)

## Selection of stocks

In [76]:
#stocks = [10909,11170,13949,18542,21792,26332,31042,60599,61532,75154,86976,89269,90199,90520,93429]
#df = df.loc[df.PERMNO.isin(stocks)]

In [117]:
filtering_len_stocks = len(df.groupby('PERMNO')['date'].nunique()[df.groupby('PERMNO')['date'].nunique() > 670].index.unique())
print(filtering_len_stocks)
stock_to_keep = df.groupby('PERMNO')['date'].nunique()[df.groupby('PERMNO')['date'].nunique() > 670].index.unique()

df = df.loc[df.PERMNO.isin(stock_to_keep)]
df

209


,date,PERMNO,SHRCD,EXCHCD,TICKER,PRC,VOL,RET,SHROUT
1378,1962-07,31077,11.0,2.0,NaN,11.25000,801.0,NaN,2208.0
1357,1962-07,30840,11.0,2.0,NaN,1.12500,331.0,NaN,6546.0
1320,1962-07,30437,14.0,2.0,NaN,17.50000,114.0,NaN,1526.0
1342,1962-07,30680,11.0,2.0,NaN,5.00000,12.0,NaN,603.0
1338,1962-07,30648,11.0,2.0,NaN,4.12500,400.0,NaN,1360.0
...,...,...,...,...,...,...,...,...,...
4217775,2019-12,19561,11.0,1.0,NaN,325.76001,1278087.0,-0.110383,562791.0
4217774,2019-12,19502,11.0,3.0,NaN,58.96000,881154.0,-0.010738,887987.0
4217773,2019-12,19393,11.0,1.0,NaN,64.19000,3339867.0,0.127327,2344185.0
4217772,2019-12,19350,11.0,1.0,NaN,173.25999,308561.0,0.035525,313276.0


## Indexing by stocks

In [114]:
df_group = df.groupby(['PERMNO','date'])[['EXCHCD', 'SHRCD', 'PRC', 'VOL', 'RET','SHROUT']].agg(sum)
df_group.reset_index(inplace= True)
print(df_group.nunique())

df_group = df.groupby(['PERMNO','date'])[['PRC', 'VOL', 'RET','SHROUT']].agg(sum)
#ERREUR DANS LA LIGNE CI DESSOUS JE VOUS EXPLIQUERAI
#df_group['monthly_returns'] = df_group.PRC.pct_change(1)
df_group['monthly_returns_12'] = df_group.PRC.pct_change(12)
#df_group['monthly_returns_prod'] = df_group['monthly_returns'].cumprod() - 1
df_group

PERMNO      231
date        690
EXCHCD        6
SHRCD        11
PRC       15933
VOL       81331
RET       80230
SHROUT    33692
dtype: int64


PRC    VOL       RET  SHROUT  monthly_returns_12
PERMNO date                                                        
10065  1962-07  24.875  167.0  0.015306  3444.0                 NaN
       1962-08  25.000  160.0  0.011055  3444.0                 NaN
       1962-09  23.500  137.0 -0.060000  3444.0                 NaN
       1962-10  23.375  205.0 -0.005319  3444.0                 NaN
       1962-11  24.750  262.0  0.108449  3444.0                 NaN
...                ...    ...       ...     ...                 ...
42614  2019-08 -81.240   78.0 -0.011404  2434.0            0.025822
       2019-09 -82.030    9.0  0.009724  2434.0           -0.067364
       2019-10  83.150   21.0  0.013654  2434.0           -2.150785
       2019-11 -87.265   23.0  0.052495  2417.0            0.002931
       2019-12 -86.660   16.0 -0.006933  4223.0            0.121595

[158486 rows x 5 columns]

I. Selectionner 10 stocks
For each stocks :
    2. Return sur 12 mois
    3. Vol de chaque stocks
    4. Risk-adjusted Price Momentum = Price Momentum / σ
    5. Compute Momentum Z-Score : z = (Risk-adjusted Price Momentum - μ) / σ
    6. The Momentum Score is then computed from the Momentum Z-Score as follows:
        𝑀𝑜𝑚𝑒𝑛𝑡𝑢𝑚 𝑆𝑐𝑜𝑟𝑒 = {
            1 + 𝑍         if  𝑍 > 0
            (1 − 𝑍)**-1   if  𝑍 < 0
            }
            
https://www.msci.com/eqb/methodology/meth_docs/MSCI_Momentum_Indexes_Methodology_May2017.pdf

https://www.investopedia.com/articles/technical/081501.asp

In [83]:
df_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4221415 entries, 0 to 4221414
Data columns (total 9 columns):
index     int64
PERMNO    int64
date      object
SHRCD     float64
EXCHCD    float64
PRC       float64
VOL       float64
RET       float64
SHROUT    float64
dtypes: float64(6), int64(2), object(1)
memory usage: 289.9+ MB
